In [1]:
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter
import numpy as np
from typing import Tuple, List, Dict
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
from matplotlib.dates import DateFormatter
import os
import joblib
from scipy.stats import median_abs_deviation
from skyfield.api import load, EarthSatellite
from datetime import datetime, timezone
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)  

Data Preparation (Merge TLE, UCS, and SatCat)

In [ ]:
def data_build_pipe(parquet, output_directory, output_filenm):
    df = pd.read_parquet(parquet)
    tle_cols = ['datetime', 'catalog_number']
    df = df.drop_duplicates(subset=tle_cols)

    ucs_df = pd.read_csv(r'C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\UCS-Satellite-Database 5-1-2023.csv',encoding='latin-1')

    with open(r"C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\CIS_satcat.pkl", 'rb') as f:
        satcat = pickle.load(f, encoding='latin1')
    satcat_df = pd.DataFrame(satcat)

    df['catalog_number'] = df['catalog_number'].astype(int)
    satcat_df['NORAD_CAT_ID'] = satcat_df['NORAD_CAT_ID'].astype(int)

    df = df.merge(
        satcat_df[['NORAD_CAT_ID', 'OBJECT_TYPE', 'SATNAME', 'SITE']], 
        left_on='catalog_number',
        right_on='NORAD_CAT_ID',
        how='left'
    )

    df = df.drop('catalog_number', axis=1)

    df = df.merge(
        ucs_df[['NORAD Number', 'Operator/Owner', 'Users', 'Purpose']], 
        left_on='NORAD_CAT_ID',
        right_on='NORAD Number',
        how='left'
    )

    df = df.drop('NORAD Number', axis=1)
    
    df.to_parquet(os.path.join(output_directory, output_filenm),engine='pyarrow', compression = 'gzip', index =True))

parquets = [r'C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\DEV_combat_tle_df.parquet',
            r'C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\DEV_inference_tle_df.parquet',
            r'C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\DEV_training_tle_df.parquet']
output_filename = ["MODEL_combat_tle_data.parquet",
                   "MODEL_inft_tle_data.parquet",
                   "MODEL_training_tle_data.parquet"]

output_dir = r"C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC"

#for f,n in zip(parquets, output_filename):
    #data_build_pipe(f, output_dir, n)


Add Outlier Column to Training Data to use as Test/Train Split and Validation

In [3]:
df = pd.read_parquet(r'C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\MODEL_training_tle_data.parquet')

In [4]:
cols = ['inclination','ra_of_asc_node', 'eccentricity', 'arg_of_perigee', 'mean_anomaly', 'mean_motion']

def calculate_outlier(df, column_nm, threshold = 2, method = 'iqr'):
    z_scores = np.abs((df[column_nm] - df[column_nm].mean())/df[column_nm].std())
    df[f'outlier_{column_nm}'] = (z_scores > threshold).astype(int)

    if method=='iqr':
        # IQR Method returned ~ 2% outlier
        q1 = df[column_nm].quantile(0.25)
        q3 = df[column_nm].quantile(0.75)
        iqr = q3 - q1
        df[f'outlier_{column_nm}'] = ((df[column_nm] < (q1 - 1.5 * iqr)) | (df[column_nm] > (q3 + 1.5 *iqr))).astype(int)
    
    elif method =='zscore':
        # ~ 5% outlier with threshold=2
        z_scores = np.abs((df[column_nm] - df[column_nm].mean())/df[column_nm].std())
        df[f'outlier_{column_nm}'] = (z_scores > threshold).astype(int)

    elif method =='mad':
        #~9% outliers with threshold=2 
        median = df[column_nm].median()
        mad = median_abs_deviation(df[column_nm])
        modified_z = 0.6745 * (df[column_nm] - median) / mad
        df[f'outlier_{column_nm}'] = (abs(modified_z) > threshold).astype(int)

    else:
        print('Not a valid outlier detection method')

    df['outlier'] = (df.iloc[:, -6:] == 1).any(axis=1).astype(int)

count = 0 
tle_df = pd.DataFrame()
for id in df['NORAD_CAT_ID'].unique():
    bysat_df = df[df['NORAD_CAT_ID']==id]
    #print(bysat_df['NORAD_CAT_ID'].unique())
    for c in cols:
        calculate_outlier(bysat_df,c, threshold = 2, method = 'mad')
    tle_df = pd.concat([tle_df, bysat_df], axis=0)
    count += 1
    progress = (count/len(df['NORAD_CAT_ID'].unique())) * 100
    print(f"Outlier calculation is {progress} percent complete")

tle_df.to_parquet(r'C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\model_test_train.parquet',engine='pyarrow', compression = 'gzip', index =True)

Outlier calculation is 0.039308176100628936 percent complete
Outlier calculation is 0.07861635220125787 percent complete
Outlier calculation is 0.1179245283018868 percent complete
Outlier calculation is 0.15723270440251574 percent complete
Outlier calculation is 0.19654088050314467 percent complete
Outlier calculation is 0.2358490566037736 percent complete
Outlier calculation is 0.2751572327044025 percent complete
Outlier calculation is 0.3144654088050315 percent complete
Outlier calculation is 0.3537735849056604 percent complete
Outlier calculation is 0.39308176100628933 percent complete
Outlier calculation is 0.43238993710691825 percent complete
Outlier calculation is 0.4716981132075472 percent complete
Outlier calculation is 0.511006289308176 percent complete
Outlier calculation is 0.550314465408805 percent complete
Outlier calculation is 0.589622641509434 percent complete
Outlier calculation is 0.628930817610063 percent complete
Outlier calculation is 0.6682389937106918 percent com

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_parquet(r'C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\MODEL_training_tle_data.parquet')

def plot_distributions(df):
    cols = ['inclination', 'ra_of_asc_node', 'eccentricity', 'arg_of_perigee', 'mean_anomaly', 'mean_motion']
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    axes = axes.flatten()  

    for i, column in enumerate(cols):
        sns.kdeplot(data=df[column], ax=axes[i], fill=True)
        axes[i].set_title(column)
        axes[i].grid(True)

    plt.tight_layout()

    plt.savefig(rf'C:\Users\dk412\Desktop\David\Python Projects\RusSat\outputdistributions_{df}.png', dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

#plot_distributions(df)

Separate Featuresa from Dataframe for Anomaly Detection

In [ ]:
NORAD_ID_NUM = 32395

samp_df = df[df['NORAD_CAT_ID']==NORAD_ID_NUM]
samp_df = samp_df.sort_values(by='datetime', ascending=False)

orb_df = samp_df[['datetime','inclination','ra_of_asc_node', 'eccentricity', 'arg_of_perigee', 'mean_anomaly', 'mean_motion']]

orb_df = orb_df.set_index('datetime', drop = True)
orb_df.info(), orb_df.head()

Anomaly Detection Model

In [ ]:
#################################################################
#               Visualization Class
#################################################################

class TLEVisualizer:
    
    @staticmethod
    def format_timestamps(timestamps, indices=None):
        if indices is not None:
            timestamps = timestamps[indices]
        if not isinstance(timestamps, pd.DatetimeIndex):
            timestamps = pd.DatetimeIndex(timestamps)
        return [ts.strftime('%Y-%m-%d %H:%M') for ts in timestamps]
    
    @staticmethod
    def plot_training_loss(losses: List[float], figsize=(10, 5)):
        plt.figure(figsize=figsize)
        plt.plot(losses, label='Training Loss')
        plt.title('Autoencoder Training Loss Over Time')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.grid(True)
        plt.legend()
        plt.show()
    
    @staticmethod
    def plot_reconstruction_errors(reconstruction_errors: np.ndarray, 
                                 feature_names: List[str],
                                 figsize=(8, 5)):
        plt.figure(figsize=figsize)
        sns.boxplot(data=reconstruction_errors)
        plt.xticks(range(len(feature_names)), feature_names, rotation=45, ha='right')
        plt.title('Distribution of Reconstruction Errors by Feature')
        plt.xlabel('Feature')
        plt.ylabel('Reconstruction Error')
        plt.tight_layout()
        plt.show()
    
    @classmethod
    def plot_anomaly_heatmap(cls, 
                            anomaly_details: Dict, 
                            feature_names: List[str],
                            timestamps,
                            max_samples: int = 50,
                            figsize=(15, 8)):
        anomalous_indices = np.where(np.any(anomaly_details['feature_anomalies'], axis=1))[0]
        
        if len(anomalous_indices) == 0:
            print("No anomalies found to visualize")
            return
        
        if len(anomalous_indices) > max_samples:
            anomalous_indices = anomalous_indices[:max_samples]
        
        z_scores = anomaly_details['z_scores'][anomalous_indices]
        time_labels = cls.format_timestamps(timestamps, anomalous_indices)
        
        plt.figure(figsize=figsize)
        plt.rcParams.update({
            'font.size': 10,
            'axes.titlesize': 12,
            'axes.labelsize': 11,
            'xtick.labelsize': 10,
            'ytick.labelsize': 8,
        })

        sns.heatmap(z_scores, 
                   xticklabels=feature_names,
                   yticklabels=time_labels,
                   cmap='RdYlBu_r',
                   center=0)
        plt.title('Anomaly Z-Scores Heatmap')
        plt.xlabel('Feature')
        plt.ylabel('Timestamp')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    
    @staticmethod
    def plot_feature_timeline(data: np.ndarray,
                            anomalies: np.ndarray,
                            feature_names: List[str],
                            timestamps,
                            n_features: int = 6,
                            figsize=(15, 12)):
        if not isinstance(timestamps, pd.DatetimeIndex):
            timestamps = pd.DatetimeIndex(timestamps)
            
        n_features = min(n_features, data.shape[1])
        
        fig = plt.figure(figsize=figsize)
        gs = GridSpec(n_features, 1, figure=fig)
        
        date_formatter = DateFormatter('%Y-%m-%d')
        
        for i in range(n_features):
            ax = fig.add_subplot(gs[i, 0])
            
            ax.plot(timestamps[~anomalies], data[~anomalies, i], 'b.', 
                   alpha=0.5, label='Normal', markersize=4)
            
            if np.any(anomalies):
                ax.plot(timestamps[anomalies], data[anomalies, i], 'r.', 
                       label='Anomaly', alpha=0.7, markersize=6)
            
            ax.set_ylabel(feature_names[i])
            ax.xaxis.set_major_formatter(date_formatter)
            ax.grid(True, alpha=0.3)
            
            if i == 0:
                ax.legend()
            if i == n_features - 1:
                ax.set_xlabel('Time')
            
            plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
        
        plt.suptitle('Feature Timeline with Anomalies')
        plt.tight_layout()
        plt.show()
    
    @classmethod
    def plot_summary(cls,
                    data: np.ndarray,
                    anomaly_details: Dict,
                    anomalies: np.ndarray,
                    feature_names: List[str],
                    timestamps,
                    losses: List[float]):
        if not isinstance(timestamps, pd.DatetimeIndex):
            timestamps = pd.DatetimeIndex(timestamps)
            
        fig = plt.figure(figsize=(20, 12))
        gs = GridSpec(2, 3, figure=fig)
        
        #Training Loss
        ax1 = fig.add_subplot(gs[0, 0])
        ax1.plot(losses)
        ax1.set_title('Training Loss')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.grid(True, alpha=0.3)
        
        #Reconstruction Error
        ax2 = fig.add_subplot(gs[0, 1:])
        sns.boxplot(data=anomaly_details['reconstruction_errors'], ax=ax2)
        ax2.set_xticklabels(feature_names, rotation=45, ha='right')
        ax2.set_title('Reconstruction Error Distribution')
        ax2.grid(True, alpha=0.3)
        
        #Anomaly Timeline
        ax3 = fig.add_subplot(gs[1, :2])
        ax3.plot(timestamps[~anomalies], np.zeros(np.sum(~anomalies)), 'b.',
                label='Normal', alpha=0.5, markersize=4)
        if np.any(anomalies):
            ax3.plot(timestamps[anomalies], np.ones(np.sum(anomalies)), 'r.',
                    label='Anomaly', alpha=0.7, markersize=6)
        ax3.set_title('Anomaly Timeline')
        ax3.set_ylabel('Anomaly (1) / Normal (0)')
        ax3.set_xlabel('Time')
        ax3.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
        ax3.grid(True, alpha=0.3)
        ax3.legend()
        plt.setp(ax3.xaxis.get_majorticklabels(), rotation=45, ha='right')
        
        #Feature Distribution
        ax4 = fig.add_subplot(gs[1, 2])
        feature_anomaly_counts = anomaly_details['feature_anomalies'].sum(axis=0)
        ax4.bar(range(len(feature_names)), feature_anomaly_counts)
        ax4.set_xticks(range(len(feature_names)))
        ax4.set_xticklabels(feature_names, rotation=45, ha='right')
        ax4.set_title('Anomalies per Feature')
        ax4.set_ylabel('Count')
        ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

#################################################################
#               Model Classes
#################################################################

class TLEEncoder(nn.Module):
    def __init__(self, input_dim: int, latent_dim: int):
        super(TLEEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, latent_dim)
        )
        
    def forward(self, x):
        return self.encoder(x)

class TLEDecoder(nn.Module):
    def __init__(self, latent_dim: int, output_dim: int):
        super(TLEDecoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )
        
    def forward(self, x):
        return self.decoder(x)

class TLEAnomalyDetector:
    def __init__(self, input_dim: int, latent_dim: int = 8):
        print(f"Initializing detector with input_dim={input_dim}, latent_dim={latent_dim}")
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        self.encoder = TLEEncoder(input_dim, latent_dim).to(self.device)
        self.decoder = TLEDecoder(latent_dim, input_dim).to(self.device)
        self.scaler = StandardScaler()
        
    def save_model(self, path: str = f"{NORAD_ID_NUM}anomaly_detector_model"):
#    def save_model(self, path: str = "anomaly_detector_model"):
        os.makedirs(path, exist_ok=True)
        
        torch.save({
            'encoder_state': self.encoder.state_dict(),
            'decoder_state': self.decoder.state_dict(),
            'input_dim': self.input_dim,
            'latent_dim': self.latent_dim
        }, os.path.join(path, f'{NORAD_ID_NUM}_model.pt'))
        
        joblib.dump(self.scaler, os.path.join(path, 'scaler.pkl'))
        
        print(f"Model saved to {path}")
    
    @classmethod
    def load_model(cls, path: str = f"{NORAD_ID_NUM}anomaly_detector_model") -> 'TLEAnomalyDetector':
        checkpoint = torch.load(os.path.join(path, f'{NORAD_ID_NUM}_model.pt'))
        
        detector = cls(input_dim=checkpoint['input_dim'], 
                     latent_dim=checkpoint['latent_dim'])
        
        detector.encoder.load_state_dict(checkpoint['encoder_state'])
        detector.decoder.load_state_dict(checkpoint['decoder_state'])
        
        detector.scaler = joblib.load(os.path.join(path, 'scaler.pkl'))
        
        detector.encoder.eval()
        detector.decoder.eval()
        
        print(f"Model loaded from {path}")
        return detector
        
    def visualize_results(self, data: np.ndarray, 
                         anomalies: np.ndarray,
                         anomaly_details: Dict,
                         feature_names: List[str],
                         timestamps: pd.DatetimeIndex,
                         losses: List[float]):
        visualizer = TLEVisualizer()
        
        print("Generating visualization summary...")
        visualizer.plot_summary(data, anomaly_details, anomalies, 
                              feature_names, timestamps, losses)
        
        print("\nGenerating detailed visualizations...")
        visualizer.plot_training_loss(losses)
        visualizer.plot_reconstruction_errors(anomaly_details['reconstruction_errors'], 
                                           feature_names)
        visualizer.plot_anomaly_heatmap(anomaly_details, feature_names, timestamps)
        visualizer.plot_feature_timeline(data, anomalies, feature_names, timestamps)
        
    def _validate_input_data(self, data: np.ndarray) -> None:
        if not isinstance(data, np.ndarray):
            raise TypeError(f"Expected numpy array, got {type(data)}")
        
        if len(data.shape) != 2:
            raise ValueError(f"Expected 2D array, got shape {data.shape}")
            
        if data.shape[1] != self.input_dim:
            raise ValueError(f"Expected {self.input_dim} features, got {data.shape[1]}")
            
        if np.isnan(data).any():
            raise ValueError("Input data contains NaN values")
            
        if np.isinf(data).any():
            raise ValueError("Input data contains infinite values")

    def train(self, data: np.ndarray, epochs: int = 150, batch_size: int = 32) -> List[float]:
        print(f"Starting training with data shape: {data.shape}")
        self._validate_input_data(data)
        
        scaled_data = self.scaler.fit_transform(data)
        train_data = torch.FloatTensor(scaled_data).to(self.device)
        
        optimizer = optim.Adam(list(self.encoder.parameters()) + 
                             list(self.decoder.parameters()))
        criterion = nn.MSELoss(reduction='none')
        losses = []
        
        n_batches = (len(train_data) + batch_size - 1) // batch_size
        print(f"Training with {n_batches} batches per epoch")
        
        for epoch in range(epochs):
            total_loss = 0
            for i in range(0, len(train_data), batch_size):
                batch = train_data[i:min(i + batch_size, len(train_data))]
                
                latent = self.encoder(batch)
                reconstructed = self.decoder(latent)
                
                loss = criterion(reconstructed, batch).mean()
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
            
            avg_loss = total_loss / n_batches
            losses.append(avg_loss)
            
            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.6f}")
                
        return losses
    
    def detect_anomalies(self, data: np.ndarray, threshold_sigma: float = 3) -> Tuple[np.ndarray, Dict]:
        print(f"Detecting anomalies in data with shape: {data.shape}")
        self._validate_input_data(data)
        
        scaled_data = self.scaler.transform(data)
        test_data = torch.FloatTensor(scaled_data).to(self.device)
        
        with torch.no_grad():
            latent = self.encoder(test_data)
            reconstructed = self.decoder(latent)
        
        reconstruction_errors = torch.abs(test_data - reconstructed)
        reconstruction_errors = reconstruction_errors.cpu().numpy()
        error_mean = np.mean(reconstruction_errors, axis=0)
        error_std = np.std(reconstruction_errors, axis=0)
        error_std = np.where(error_std == 0, 1e-10, error_std)
        
        z_scores = (reconstruction_errors - error_mean) / error_std
        anomalies = np.any(np.abs(z_scores) > threshold_sigma, axis=1)
        
        anomaly_details = {
            'z_scores': z_scores,
            'feature_anomalies': np.abs(z_scores) > threshold_sigma,
            'reconstruction_errors': reconstruction_errors
        }
        
        print(f"Found {np.sum(anomalies)} anomalous samples")
        return anomalies, anomaly_details
    
    def explain_anomalies(self, anomaly_details: Dict, feature_names: List[str] = None) -> List[Dict]:
        if feature_names is None:
            feature_names = [f"Feature_{i}" for i in range(anomaly_details['z_scores'].shape[1])]
        
        if len(feature_names) != anomaly_details['z_scores'].shape[1]:
            raise ValueError(f"Expected {anomaly_details['z_scores'].shape[1]} feature names, got {len(feature_names)}")
            
        print(f"Generating explanations for shape {anomaly_details['z_scores'].shape}")
        explanations = []
        
        for i in range(len(anomaly_details['z_scores'])):
            if np.any(anomaly_details['feature_anomalies'][i]):
                anomalous_features = []
                for j, is_anomaly in enumerate(anomaly_details['feature_anomalies'][i]):
                    if is_anomaly:
                        anomalous_features.append({
                            'feature': feature_names[j],
                            'z_score': float(anomaly_details['z_scores'][i, j]),  
                            'reconstruction_error': float(anomaly_details['reconstruction_errors'][i, j])
                        })
                
                if anomalous_features:
                    explanations.append({
                        'sample_index': int(i),  
                        'anomalous_features': anomalous_features
                    })
                    
        print(f"Generated {len(explanations)} anomaly explanations")
        return explanations

def run_anomaly_detection_pipeline(df: pd.DataFrame, 
                                 feature_names: List[str] = None,
                                 model_path: str = None,
                                 should_train: bool = False):

    try:
        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)
            
        data = df.values
        timestamps = df.index
        input_dim = data.shape[1]
        
        if should_train:
            print("Training new model...")
            detector = TLEAnomalyDetector(input_dim=input_dim)
            losses = detector.train(data, epochs=150)
            
            if model_path:
                detector.save_model(model_path)
        else:
            if not model_path:
                raise ValueError("model_path must be provided when should_train=False")
            print("Loading existing model...")
            detector = TLEAnomalyDetector.load_model(model_path)
        
        print("Detecting anomalies...")
        anomalies, anomaly_details = detector.detect_anomalies(data, 3)
        
        if feature_names is None:
            feature_names = df.columns.tolist()
            
        print("Generating explanations...")
        explanations = detector.explain_anomalies(anomaly_details, feature_names)
        
        print("Generating visualizations...")
        detector.visualize_results(data, anomalies, anomaly_details, 
                                 feature_names, timestamps, 
                                 losses if should_train else [])
        
        return detector, anomalies, explanations, timestamps, anomaly_details
        
    except Exception as e:
        print(f"Error during anomaly detection: {str(e)}")
        raise

In [ ]:
feature_names = list(orb_df)

detector, anomalies, explanations, timestamps, anomaly_details = run_anomaly_detection_pipeline(
    orb_df,
    feature_names=feature_names,
    model_path=r"C:\Users\dk412\Desktop\David\Python Projects\RusSat\anomaly_model",
    should_train=True
)

anom_dts_idx = []
anom_dict = {}
print(f"\nFound {sum(anomalies)} anomalous samples")
for exp in explanations:
    print(f"\nAnomaly at sample {exp['sample_index']} on {orb_df.index[exp['sample_index']].strftime('%Y-%m-%d')}:")
    anom_dts_idx.append(exp['sample_index'])

    feature_lst =[]

    for feat in exp['anomalous_features']:
        print(f"- {feat['feature']}: z-score = {feat['z_score']:.2f}")
        feature_lst.append(feat['feature'])

    anom_dict[exp['sample_index']] = feature_lst

Add Anomaly Info to Dataframe

In [ ]:
#samp_df['anomaly'] = [1 if i in anom_dts_idx else 0 for i in range(len(samp_df))]
full_df = samp_df.copy(deep=False)
#full_df['date'] = full_df.index
#full_df['date'] = pd.to_datetime(full_df['date']).dt.date
full_df.reset_index(inplace=True, drop = True)

all_features = set()
for features_list in anom_dict.values():
    all_features.update(features_list)

anom_df = pd.DataFrame(0, 
                      index=anom_dict.keys(),
                      columns=[f'anom_{feat}' for feat in sorted(all_features)])

for key, features in anom_dict.items():
    anom_df.loc[key, [f'anom_{feat}' for feat in features]] = 1

full_df = pd.concat([full_df, anom_df], axis=1)

anom_cols = full_df.columns.str.startswith('anom')
anom_data = full_df.iloc[:, anom_cols]
full_df['anom_count'] = anom_data.sum(axis=1)

full_df = full_df.fillna(0)

In [ ]:
full_df.iloc[90:95]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scattergeo(
    lon=full_df['x'],  
    lat=full_df['y'],  
    mode='markers',
    marker=dict(
        size=5,
        color=['red' if count > 0 else 'blue' for count in full_df['anom_count']],
        opacity=0.8,
        symbol='circle'
    ),
    text=full_df['anom_count'].apply(lambda x: f'Anomaly Count: {x}'),
    name='Satellites'
))

fig.update_layout(
    title='Global Satellite Positions',
    geo=dict(
        projection_type='orthographic',  # 3D globe view
        showland=True,
        showocean=True,
        showcoastlines=True,
        showcountries=True,
        oceancolor='rgb(204, 229, 255)',
        landcolor='rgb(243, 243, 243)',
        coastlinecolor='rgb(80, 80, 80)',
        countrycolor='rgb(80, 80, 80)',
        bgcolor='rgba(255, 255, 255, 0)',
        framecolor='rgba(255, 255, 255, 0)',
        showframe=False
    ),
    paper_bgcolor='rgba(255, 255, 255, 1)',
    plot_bgcolor='rgba(255, 255, 255, 1)',
    margin=dict(l=0, r=0, t=30, b=0)
)

fig.show()

Inference

In [ ]:
combat_data = pd.read_parquet(r"C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\DEV_combat_tle_df.parquet")
tle_cols = ['datetime', 'catalog_number']
combat_data = combat_data.drop_duplicates(subset=tle_cols)

In [ ]:
combat_data['datetime'].max()

In [ ]:
combat_data['catalog_number'] = combat_data['catalog_number'].astype(int)
satcat_df['NORAD_CAT_ID'] = satcat_df['NORAD_CAT_ID'].astype(int)

combat_data = combat_data.merge(
    satcat_df[['NORAD_CAT_ID', 'OBJECT_TYPE', 'SATNAME', 'SITE']], 
    left_on='catalog_number',
    right_on='NORAD_CAT_ID',
    how='left'
)

combat_data = combat_data.drop('catalog_number', axis=1)

combat_data = combat_data.merge(
    ucs_df[['NORAD Number', 'Operator/Owner', 'Users', 'Purpose']], 
    left_on='NORAD_CAT_ID',
    right_on='NORAD Number',
    how='left'
)

combat_data = combat_data.drop('NORAD Number', axis=1)

In [ ]:
combat_samp_df = combat_data[combat_data['NORAD_CAT_ID']==NORAD_ID_NUM]
combat_samp_df = combat_samp_df.sort_values(by='datetime', ascending=False)

combat_orb_df = combat_samp_df[['datetime','mean_motion_dot','mean_motion_ddot','inclination','ra_of_asc_node', 'eccentricity', 'arg_of_perigee', 'mean_anomaly', 'mean_motion']]

combat_orb_df = combat_orb_df.set_index('datetime', drop = True)
combat_orb_df.info(), combat_orb_df.head()

In [ ]:
feature_names = list(combat_orb_df)

# Load and use on new data
detector, anomalies, explanations, timestamps, anomaly_details = run_anomaly_detection_pipeline(
    combat_orb_df,
    feature_names=feature_names,
    model_path=r"C:\Users\dk412\Desktop\David\Python Projects\RusSat\anomaly_model",
    should_train=False
)
 
anom_dts_idx = []
anom_dict = {}
print(f"\nFound {sum(anomalies)} anomalous samples")
for exp in explanations:
    print(f"\nAnomaly at sample {exp['sample_index']} on {combat_orb_df.index[exp['sample_index']].strftime('%Y-%m-%d')}:")
    anom_dts_idx.append(exp['sample_index'])

    feature_lst =[]

    for feat in exp['anomalous_features']:
        print(f"- {feat['feature']}: z-score = {feat['z_score']:.2f}")
        feature_lst.append(feat['feature'])

    anom_dict[exp['sample_index']] = feature_lst

In [ ]:
#samp_df['anomaly'] = [1 if i in anom_dts_idx else 0 for i in range(len(samp_df))]
combat_full_df = combat_samp_df.copy(deep=False)
#combat_full_df['date'] = combat_full_df.index
#combat_full_df['date'] = pd.to_datetime(combat_full_df['date']).dt.date
combat_full_df.reset_index(inplace=True, drop = True)

all_features = set()
for features_list in anom_dict.values():
    all_features.update(features_list)

combat_anom_df = pd.DataFrame(0, 
                      index=anom_dict.keys(),
                      columns=[f'anom_{feat}' for feat in sorted(all_features)])

for key, features in anom_dict.items():
    combat_anom_df.loc[key, [f'anom_{feat}' for feat in features]] = 1

combat_full_df = pd.concat([combat_full_df, combat_anom_df], axis=1)

anom_cols = combat_full_df.columns.str.startswith('anom')
anom_data = combat_full_df.iloc[:, anom_cols]
combat_full_df['anom_count'] = anom_data.sum(axis=1)

combat_full_df = combat_full_df.fillna(0)

In [ ]:
anomalies_df = combat_full_df.loc[combat_full_df['anom_count'] > 0]

In [ ]:
anomalies_df['anom_count'].value_counts()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scattergeo(
    lon=anomalies_df['x'],  
    lat=anomalies_df['y'],  
    mode='markers',
    marker=dict(
        size=5,
        color=['red' if count > 0 else 'blue' for count in anomalies_df['anom_count']],
        opacity=0.8,
        symbol='circle'
    ),
    text=full_df['anom_count'].apply(lambda x: f'Anomaly Count: {x}'),
    name='Satellites'
))

fig.update_layout(
    title='Global Satellite Positions',
    geo=dict(
        projection_type='orthographic',  # 3D globe view
        showland=True,
        showocean=True,
        showcoastlines=True,
        showcountries=True,
        oceancolor='rgb(204, 229, 255)',
        landcolor='rgb(243, 243, 243)',
        coastlinecolor='rgb(80, 80, 80)',
        countrycolor='rgb(80, 80, 80)',
        bgcolor='rgba(255, 255, 255, 0)',
        framecolor='rgba(255, 255, 255, 0)',
        showframe=False
    ),
    paper_bgcolor='rgba(255, 255, 255, 1)',
    plot_bgcolor='rgba(255, 255, 255, 1)',
    margin=dict(l=0, r=0, t=30, b=0)
)

fig.show()

In [ ]:
##################################################################################################################
#                                            PROD CODE - ANOM DETECT
###################################################################################################################

import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter
import numpy as np
from typing import Tuple, List, Dict
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
from matplotlib.dates import DateFormatter
import os
import joblib
import plotly.graph_objects as go
import sys
sys.path.append(r'C:\Users\dk412\Desktop\David\Python Projects\RusSat\prod_code')
from helpers import *
from skyfield.api import load, EarthSatellite
from datetime import datetime, timezone
import numpy as np


#################################################################
#               Model Save and Load
#################################################################

def save_model(self, path: str = f"{e}_anomaly_detector_model"):
#    def save_model(self, path: str = "anomaly_detector_model"):
    os.makedirs(path, exist_ok=True)
    
    torch.save({
        'encoder_state': self.encoder.state_dict(),
        'decoder_state': self.decoder.state_dict(),
        'input_dim': self.input_dim,
        'latent_dim': self.latent_dim
    }, os.path.join(path, f'{e}_model.pt'))
    
    joblib.dump(self.scaler, os.path.join(path, 'scaler.pkl'))
    
    print(f"Model saved to {path}")

@classmethod
def load_model(cls, path: str = f"{e}_anomaly_detector_model") -> 'TLEAnomalyDetector':
    checkpoint = torch.load(os.path.join(path, f'{e}_model.pt'))
    
    detector = cls(input_dim=checkpoint['input_dim'], 
                    latent_dim=checkpoint['latent_dim'])
    
    detector.encoder.load_state_dict(checkpoint['encoder_state'])
    detector.decoder.load_state_dict(checkpoint['decoder_state'])
    
    detector.scaler = joblib.load(os.path.join(path, 'scaler.pkl'))
    
    detector.encoder.eval()
    detector.decoder.eval()
    
    print(f"Model loaded from {path}")
    return detector
        

##################################################################
#                      Data Preprocessing
###################################################################

def data_build_pipeline(parquet_file):
    df = pd.read_parquet(parquet_file)
    tle_cols = ['datetime', 'catalog_number']
    df = df.drop_duplicates(subset=tle_cols)

    ucs_df = pd.read_csv(r'C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\UCS-Satellite-Database 5-1-2023.csv',encoding='latin-1')

    with open(r"C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\CIS_satcat.pkl", 'rb') as f:
        satcat = pickle.load(f, encoding='latin1')
    
    satcat_df = pd.DataFrame(satcat)

    df['catalog_number'] = df['catalog_number'].astype(int)
    satcat_df['NORAD_CAT_ID'] = satcat_df['NORAD_CAT_ID'].astype(int)

    df = df.merge(
        satcat_df[['NORAD_CAT_ID', 'OBJECT_TYPE', 'SATNAME', 'SITE']], 
        left_on='catalog_number',
        right_on='NORAD_CAT_ID',
        how='left'
    )

    df = df.drop('catalog_number', axis=1)

    df = df.merge(
        ucs_df[['NORAD Number', 'Operator/Owner', 'Users', 'Purpose']], 
        left_on='NORAD_CAT_ID',
        right_on='NORAD Number',
        how='left'
    )

    df = df.drop('NORAD Number', axis=1)

    return df

##################################################################
#                      Model Train and Fetch
###################################################################
def develop_fetch_model(df, NORAD_ID_NUM, model_training):
    samp_df = df[df['NORAD_CAT_ID']==NORAD_ID_NUM]
    samp_df = samp_df.sort_values(by='datetime', ascending=False)

    orb_df = samp_df[['datetime','mean_motion_dot','mean_motion_ddot','inclination','ra_of_asc_node', 'eccentricity', 'arg_of_perigee', 'mean_anomaly', 'mean_motion']]

    orb_df = orb_df.set_index('datetime', drop = True)
    orb_df.info(), orb_df.head()

    feature_names = list(orb_df)

    if model_training:
        detector, anomalies, explanations, timestamps, anomaly_details = run_anomaly_detection_pipeline(
            orb_df,
            feature_names=feature_names,
            model_path=r"C:\Users\dk412\Desktop\David\Python Projects\RusSat\anomaly_model",
            should_train=True
        )
    else:
        detector, anomalies, explanations, timestamps, anomaly_details = run_anomaly_detection_pipeline(
            orb_df,
            feature_names=feature_names,
            model_path=r"C:\Users\dk412\Desktop\David\Python Projects\RusSat\anomaly_model",
            should_train=False
        )

    anom_dts_idx = []
    anom_dict = {}
    print(f"\nFound {sum(anomalies)} anomalous samples")
    for exp in explanations:
        print(f"\nAnomaly at sample {exp['sample_index']} on {orb_df.index[exp['sample_index']].strftime('%Y-%m-%d')}:")
        anom_dts_idx.append(exp['sample_index'])

        feature_lst =[]

        for feat in exp['anomalous_features']:
            print(f"- {feat['feature']}: z-score = {feat['z_score']:.2f}")
            feature_lst.append(feat['feature'])

        anom_dict[exp['sample_index']] = feature_lst

    full_df = samp_df.copy(deep=False)
    full_df.reset_index(inplace=True, drop = True)

    all_features = set()
    for features_list in anom_dict.values():
        all_features.update(features_list)

    anom_df = pd.DataFrame(0, 
                        index=anom_dict.keys(),
                        columns=[f'anom_{feat}' for feat in sorted(all_features)])

    for key, features in anom_dict.items():
        anom_df.loc[key, [f'anom_{feat}' for feat in features]] = 1

    full_df = pd.concat([full_df, anom_df], axis=1)

    anom_cols = full_df.columns.str.startswith('anom')
    anom_data = full_df.iloc[:, anom_cols]
    full_df['anom_count'] = anom_data.sum(axis=1)

    full_df = full_df.fillna(0)

    fig = go.Figure()

    fig.add_trace(go.Scattergeo(
        lon=full_df['x'],  
        lat=full_df['y'],  
        mode='markers',
        marker=dict(
            size=5,
            color=['red' if count > 0 else 'blue' for count in full_df['anom_count']],
            opacity=0.8,
            symbol='circle'
        ),
        text=full_df['anom_count'].apply(lambda x: f'Anomaly Count: {x}'),
        name='Satellites'
    ))

    fig.update_layout(
        title='Training Data Satellite Positions',
        geo=dict(
            projection_type='orthographic',  # 3D globe view
            showland=True,
            showocean=True,
            showcoastlines=True,
            showcountries=True,
            oceancolor='rgb(204, 229, 255)',
            landcolor='rgb(243, 243, 243)',
            coastlinecolor='rgb(80, 80, 80)',
            countrycolor='rgb(80, 80, 80)',
            bgcolor='rgba(255, 255, 255, 0)',
            framecolor='rgba(255, 255, 255, 0)',
            showframe=False
        ),
        paper_bgcolor='rgba(255, 255, 255, 1)',
        plot_bgcolor='rgba(255, 255, 255, 1)',
        margin=dict(l=0, r=0, t=30, b=0)
    )

    fig.show()

    return full_df

In [ ]:
df = data_build_pipeline(r"C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\DEV_training_tle_df.parquet")
inf_df = data_build_pipeline(r"C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\DEV_combat_tle_df.parquet")

results_train_df = pd.DataFrame()
results_inf_df = pd.DataFrame()

for e in df['NORAD_CAT_ID'].unique():
    full_train_df = develop_training_model(df, e, True)
    full_inf_df = develop_training_model(inf_df, e, False)
    results_train_df = pd.concat([results_train_df, full_train_df], ignore_index = True)
    print("\nresults_train_df:\t", results_train_df.shape)
    results_inf_df = pd.concat([results_inf_df, full_inf_df], ignore_index = True)
    print("\nresults_inf_df:\t", results_inf_df.shape)
